#  單因子變異數分析 - Python實戰：如何決定多廣告的優化策略？

#### 作者：徐子皓(臺灣行銷研究特邀作者)
#### 完整文章介紹鏈接：

## 一、事前檢驗

### 1. shapiro常態性檢定

In [7]:
import scipy.stats as st
st.shapiro(alist)

ShapiroResult(statistic=0.9915865659713745, pvalue=0.6819638013839722)

In [8]:
st.shapiro(blist)

ShapiroResult(statistic=0.9929247498512268, pvalue=0.805463969707489)

In [9]:
st.shapiro(clist)

ShapiroResult(statistic=0.9802003502845764, pvalue=0.07433854043483734)

### 2. levene同質性檢定

In [10]:
st.levene(alist, blist, clist, center='mean')

LeveneResult(statistic=0.6891243668404422, pvalue=0.5026819827621525)

## 二、變異數分析

In [11]:
f_value, p_value = st.f_oneway(alist, blist, clist)
p_value

0.0955577857122512

## 三、事後檢定

### 1. pairwise_tukey事後檢定

In [13]:
from pingouin import pairwise_tukey
m_comp = pairwise_tukey(data=data, dv='消費金額', between='廣告')
m_comp

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,廣告1,廣告2,17.135167,15.592417,1.54275,0.781281,1.974642,0.118914,0.254121
1,廣告1,廣告3,17.135167,15.748167,1.38700,0.781281,1.775290,0.178294,0.228466
2,廣告2,廣告3,15.592417,15.748167,-0.15575,0.781281,-0.199352,0.900000,-0.025655


### 2. 整理成可參考報表

In [15]:
table = m_comp.drop(columns = ['mean(A)', 'mean(B)', 'T', 'p-tukey', 'hedges'])
table

,A,B,diff,se
0,廣告1,廣告2,1.54275,0.781281
1,廣告1,廣告3,1.38700,0.781281
2,廣告2,廣告3,-0.15575,0.781281


## 四、進階資料處理

### 1. 計算反轉資料

In [17]:
# 「A」欄反轉資料
add_A = table['B'].tolist()
# 「B」欄反轉資料
add_B = table['A'].tolist()
# 「diff」欄反轉資料
diff =  (table['diff'] - 2 * table['diff']).tolist()
# 「se」欄反轉資料
se = table['se'].tolist()
# 將反轉資料合併
table2 = pd.DataFrame(zip(add_A, add_B, diff, se), columns = ['A', 'B', 'diff', 'se'])

In [18]:
table2

,A,B,diff,se
0,廣告2,廣告1,-1.54275,0.781281
1,廣告3,廣告1,-1.38700,0.781281
2,廣告3,廣告2,0.15575,0.781281


### 2. 將資料整併成new_table

In [19]:
new_table = pd.concat([table, table2], ignore_index=True)
new_table

,A,B,diff,se
0,廣告1,廣告2,1.54275,0.781281
1,廣告1,廣告3,1.38700,0.781281
2,廣告2,廣告3,-0.15575,0.781281
3,廣告2,廣告1,-1.54275,0.781281
4,廣告3,廣告1,-1.38700,0.781281
5,廣告3,廣告2,0.15575,0.781281


### 3. 計算資料上下界

In [20]:
new_table['上界'] = new_table['diff'] + new_table['se']*1.96
new_table['下界'] = new_table['diff'] - new_table['se']*1.96
new_table

,A,B,diff,se,上界,下界
0,廣告1,廣告2,1.54275,0.781281,3.074061,0.011439
1,廣告1,廣告3,1.38700,0.781281,2.918311,-0.144311
2,廣告2,廣告3,-0.15575,0.781281,1.375561,-1.687061
3,廣告2,廣告1,-1.54275,0.781281,-0.011439,-3.074061
4,廣告3,廣告1,-1.38700,0.781281,0.144311,-2.918311
5,廣告3,廣告2,0.15575,0.781281,1.687061,-1.375561


### 4. np.sign用法示意

In [21]:
import numpy as np
np.sign(12)

1

In [22]:
np.sign(-5.6)

-1.0

### 5. 判斷資料是否顯著

In [23]:
justice = []
for i in range(0,new_table.shape[0]):
    a = np.sign(new_table.iloc[i,4]) # 上界正負數判斷
    b = np.sign(new_table.iloc[i,5]) # 下界正負數判斷
    if a == b:
        justice.append('Yes')
    else:
        justice.append('No')

In [24]:
justice

['Yes', 'No', 'No', 'Yes', 'No', 'No']

### 6. 將「是否顯著」結果回存至new_table中

In [25]:
new_table['是否顯著'] = justice
new_table

,A,B,diff,se,上界,下界,是否顯著
0,廣告1,廣告2,1.54275,0.781281,3.074061,0.011439,Yes
1,廣告1,廣告3,1.38700,0.781281,2.918311,-0.144311,No
2,廣告2,廣告3,-0.15575,0.781281,1.375561,-1.687061,No
3,廣告2,廣告1,-1.54275,0.781281,-0.011439,-3.074061,Yes
4,廣告3,廣告1,-1.38700,0.781281,0.144311,-2.918311,No
5,廣告3,廣告2,0.15575,0.781281,1.687061,-1.375561,No


## 五、資料視覺化	

### 1. 基本資料輸入

In [26]:
import plotly.offline as py
import plotly.graph_objects as go
fig = go.Figure()
for i in range(0, new_table.shape[0]):
    if new_table.iloc[i,6] == 'Yes':
        color = 'firebrick'
        name = '顯著'
    else:
        color = 'green'
        name = '不顯著'
    fig.add_trace(go.Scatter(
        x = [new_table.iloc[i,5], new_table.iloc[i,2], new_table.iloc[i,4]],
        y = [new_table.iloc[i,0] + '-' + new_table.iloc[i,1], new_table.iloc[i,0] + '-' + new_table.iloc[i,1], new_table.iloc[i,0] + '-' + new_table.iloc[i,1]],
        mode = "lines+markers",
        textfont=dict(
            family="sans serif",
            size=16,
            color=color),    
            line=dict(color=color, width=2),
        name = name,
        legendgroup = name,
        ))

### 2. 設定佈景主題

In [27]:
fig.update_layout(
    title={
        'text': "<b>One-Way ANOVA 廣告效益分析</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',},
    width=1800,
    height=960,
    boxmode='group',
    font=dict(
        family="Courier New, monospace",
        size=20,
        color="lightslategrey"
    )
    )   

### 3. 將產出圖片另存

In [ ]:
# 另存互動式網頁
py.plot(fig, filename='One-Way ANOVA 廣告效益分析', auto_open=True)
# 另存.png圖檔
fig.write_image("C:/Users/user/Desktop/單因子變異數分析1.5/One-Way ANOVA 廣告效益分析.png")